In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.text.all import *

In [3]:
def hook_fn(m, i, o):
  print(f"Working for layer: -- {m._get_name()} --")

awd_lstm_modified =  AWD_LSTM_M(vocab_sz=3,
                  emb_sz=5,
                  n_hid=6,
                  n_layers=2)

awd_lstm_original =  AWD_LSTM(vocab_sz=3,
                  emb_sz=5,
                  n_hid=6,
                  n_layers=2)

change none


In [8]:
flatten_model(awd_lstm_original)

[Embedding(3, 5, padding_idx=1),
 Embedding(3, 5, padding_idx=1),
 LSTM(5, 6, batch_first=True),
 ParameterModule(),
 LSTM(6, 5, batch_first=True),
 ParameterModule(),
 RNNDropout(),
 RNNDropout(),
 RNNDropout()]

In [7]:
flatten_model(awd_lstm_modified)

[Embedding(3, 5, padding_idx=1),
 LSTM(5, 6, batch_first=True),
 LSTM(6, 5, batch_first=True),
 RNNDropout(),
 RNNDropout(),
 RNNDropout()]

In [9]:
awd_lstm_modified

AWD_LSTM_M(
  (encoder): Embedding(3, 5, padding_idx=1)
  (encoder_dp): Embedding(3, 5, padding_idx=1)
  (rnns): ModuleList(
    (0): LSTM(5, 6, batch_first=True)
    (1): LSTM(6, 5, batch_first=True)
  )
  (input_dp): RNNDropout()
  (hidden_dps): ModuleList(
    (0): RNNDropout()
    (1): RNNDropout()
  )
)

In [10]:
awd_lstm_original

AWD_LSTM(
  (encoder): Embedding(3, 5, padding_idx=1)
  (encoder_dp): EmbeddingDropout(
    (emb): Embedding(3, 5, padding_idx=1)
  )
  (rnns): ModuleList(
    (0): WeightDropout(
      (module): LSTM(5, 6, batch_first=True)
    )
    (1): WeightDropout(
      (module): LSTM(6, 5, batch_first=True)
    )
  )
  (input_dp): RNNDropout()
  (hidden_dps): ModuleList(
    (0): RNNDropout()
    (1): RNNDropout()
  )
)

In [91]:
flatten_model(awd_lstm_original)

[Embedding(3, 5, padding_idx=1),
 Embedding(3, 5, padding_idx=1),
 LSTM(5, 6, batch_first=True),
 ParameterModule(),
 LSTM(6, 5, batch_first=True),
 ParameterModule(),
 RNNDropout(),
 RNNDropout(),
 RNNDropout()]

In [11]:
for model in [awd_lstm_modified, awd_lstm_original]:
    print('-------')
    counter = 0
    for m in flatten_model(model):
        counter += 1
        if has_params(m):
            print(m._get_name(), str(counter))
            m.register_forward_hook(hook_fn)

-------
Embedding 1
LSTM 2
LSTM 3
-------
Embedding 1
Embedding 2
LSTM 3
ParameterModule 4
LSTM 5
ParameterModule 6


In [5]:
awd_lstm_modified(torch.randint(3, (1,4)))

Working for layer: -- Embedding --
Working for layer: -- LSTM --
Working for layer: -- LSTM --


tensor([[[-0.0569,  0.1210,  0.0122,  0.0238, -0.0678],
         [-0.0847,  0.1975,  0.0255,  0.0458, -0.0901],
         [-0.0953,  0.2432,  0.0356,  0.0623, -0.0983],
         [-0.0981,  0.2700,  0.0421,  0.0735, -0.1016]]],
       grad_fn=<TransposeBackward0>)

In [6]:
awd_lstm_original(torch.randint(3, (1,4)))

tensor([[[ 0.0673,  0.0205, -0.0193, -0.0155,  0.1494],
         [ 0.1281,  0.0313, -0.0305, -0.0305,  0.2006],
         [ 0.1716,  0.0351, -0.0360, -0.0394,  0.2196],
         [ 0.2009,  0.0367, -0.0372, -0.0435,  0.2275]]],
       grad_fn=<TransposeBackward0>)

#### Test the objects for _forward_hooks

In [34]:
type(awd_lstm_modified)

fastai.text.models.awdlstm.AWD_LSTM_M

In [42]:
awd_lstm_modified.rnns[0]._forward_hooks

OrderedDict([(1, <function __main__.hook_fn(m, i, o)>),
             (10, <function __main__.hook_fn(m, i, o)>)])

In [43]:
awd_lstm_original.rnns[0]._forward_hooks

OrderedDict()

In [60]:
awd_lstm_original.rnns[0].module._forward_hooks

OrderedDict([(5, <function __main__.hook_fn(m, i, o)>),
             (14, <function __main__.hook_fn(m, i, o)>)])

In [61]:
awd_lstm_original.rnns[0].module._forward_hooks

OrderedDict([(5, <function __main__.hook_fn(m, i, o)>),
             (14, <function __main__.hook_fn(m, i, o)>)])

#### This is showing when the hooks get called
When it WeightedDropout.forward() - no hooks called

When WeightedDropout\['module'].forward() is called - yes hooks called.

In [79]:
x_1 = torch.randint(3,(1,4))
x_2 = awd_lstm_original.encoder(x_1)

Working for layer: -- Embedding --
Working for layer: -- Embedding --
Working for layer: -- Embedding --
Working for layer: -- Embedding --


In [87]:
print(awd_lstm_original.rnns[0])
print('----')

x_3 = awd_lstm_original.rnns[0](x_2) #.module

WeightDropout(
  (module): LSTM(5, 6, batch_first=True)
)
----


In [88]:
print(awd_lstm_original.rnns[0].module)
print('----')

x_3 = awd_lstm_original.rnns[0].module(x_2) #.module

LSTM(5, 6, batch_first=True)
----
Working for layer: -- LSTM --
Working for layer: -- LSTM --


torch.Size([1, 4, 5])